### Assignment-3
K N Anantha nandanan
AM.EN.U4CSE19326

### Imports

In [46]:
import nltk
import pandas as pd
import os
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from pprint import pprint
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

### Load the Dataset

In [6]:
path = '../Datas/20newsgroup.csv'
df = pd.read_csv(path)
df.head()

,Unnamed: 0,text,categories
0,18004,From: damien@b63519.student.cwru.edu (Damien N...,comp_sys_ibm_pc_hardware
1,9185,From: an030@cleveland.Freenet.Edu (Broward Hor...,talk_politics_misc
2,8725,From: lee139@gaul.csd.uwo.ca (Steve Lee) Subje...,rec_sport_hockey
3,7805,From: dfs@doe.carleton.ca (David F. Skoll) Sub...,talk_politics_mideast
4,8592,From: cathy@LANCE.ColoState.Edu (Cathy Smith) ...,talk_politics_guns


### Preprocessing

In [7]:
def preProcessing(text):
    #print(type(text))
    stemm_snow = SnowballStemmer("english")
    lem = WordNetLemmatizer()
    text = text.replace("\n", " ")
    text = re.sub(r"[0-9]", "", text)
    text = text.lower()
    text = word_tokenize(text)
    #text = [stemm_snow.stem(word) for word in text]
    text = [lem.lemmatize(word) for word in text]
    text = [char for char in text if char not in string.punctuation]
    text = [word for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    
    return text

### LDA Model

In [8]:
df['clean_text'] = df['text'].apply(preProcessing)

In [11]:
df.head()

,Unnamed: 0,text,categories,clean_text
0,18004,From: damien@b63519.student.cwru.edu (Damien N...,comp_sys_ibm_pc_hardware,damien b.student.cwru.edu damien neil subject ...
1,9185,From: an030@cleveland.Freenet.Edu (Broward Hor...,talk_politics_misc,cleveland.freenet.edu broward horne subject wa...
2,8725,From: lee139@gaul.csd.uwo.ca (Steve Lee) Subje...,rec_sport_hockey,lee gaul.csd.uwo.ca steve lee subject stop may...
3,7805,From: dfs@doe.carleton.ca (David F. Skoll) Sub...,talk_politics_mideast,dfs doe.carleton.ca david f. skoll subject isr...
4,8592,From: cathy@LANCE.ColoState.Edu (Cathy Smith) ...,talk_politics_guns,cathy lance.colostate.edu cathy smith subject ...


In [16]:
docs =[]
for d in df.clean_text:
    docs.append(d.split())

In [17]:
docs[0]

['damien',
 'b.student.cwru.edu',
 'damien',
 'neil',
 'subject',
 'hot',
 'cpu',
 'organization',
 'case',
 'western',
 'reserve',
 'university',
 'cleveland',
 'ohio',
 'usa',
 'line',
 'distribution',
 'na',
 'nntp-posting-host',
 'b.student.cwru.edu',
 'x-newsreader',
 'tin',
 'version',
 'pl',
 'christopher',
 'kushmerick',
 'kushmer',
 'bnlux.bnl.gov',
 'wrote',
 'hot',
 'cpu',
 'dx',
 'machine',
 'currently',
 'get',
 'hot',
 'hold',
 'finger',
 's.',
 'seem',
 'recall',
 'run',
 'somewhere',
 'close',
 'boiling',
 'point',
 'water',
 'anyone',
 'exact',
 'temperature',
 'anyway',
 'putting',
 'cpu',
 'fan/heat',
 'sink',
 'wo',
 "n't",
 'hurt',
 'could',
 'help',
 'depends',
 'paranoid',
 '...',
 '--',
 'damien',
 'neil',
 'dpn',
 'po.cwru.edu',
 '``',
 'someone',
 'debugs',
 'reality',
 'best',
 'case',
 'western',
 'reserve',
 'university',
 'quick',
 'patch',
 "''",
 'cmps/eeap',
 'linux',
 '--',
 'choice',
 'gnu',
 'generation',
 '-erik',
 'green']

In [18]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
# Filter out words that occur less than 20 documents, or more than 50% of the doc
dictionary.filter_extremes(no_below=20, no_above=0.5,  keep_n=100000)

In [19]:
## Create the doc - Bag of Words
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]
bow_corpus[4000]

[(0, 2),
 (2, 2),
 (7, 1),
 (32, 1),
 (38, 1),
 (53, 1),
 (73, 1),
 (79, 1),
 (106, 1),
 (120, 1),
 (121, 1),
 (139, 1),
 (194, 2),
 (201, 1),
 (204, 1),
 (215, 1),
 (233, 1),
 (268, 1),
 (281, 1),
 (297, 1),
 (378, 1),
 (407, 1),
 (441, 1),
 (466, 1),
 (468, 1),
 (476, 1),
 (628, 1),
 (689, 1),
 (702, 1),
 (764, 1),
 (830, 1),
 (1032, 2),
 (1061, 1),
 (1108, 1),
 (1133, 1),
 (1149, 1),
 (1212, 1),
 (1331, 4),
 (1543, 1),
 (1609, 1),
 (1650, 1),
 (1672, 1),
 (1830, 1),
 (1856, 1),
 (2001, 1),
 (2063, 1),
 (2112, 1),
 (2182, 2),
 (2204, 1),
 (2265, 1),
 (2310, 1),
 (2354, 2),
 (2757, 1),
 (2808, 1),
 (2880, 1),
 (2917, 1),
 (3006, 1),
 (3010, 2),
 (3133, 1),
 (3164, 1),
 (3475, 1),
 (3645, 1),
 (3646, 1),
 (3657, 1),
 (3673, 3),
 (3891, 1),
 (3918, 1),
 (4372, 1),
 (4542, 1),
 (4945, 1),
 (5145, 1),
 (5161, 1),
 (5383, 1),
 (6107, 1),
 (6230, 1),
 (6479, 1),
 (6642, 1),
 (6647, 1),
 (8846, 1),
 (10016, 1),
 (10654, 1)]

In [21]:
## Building the LDA model
model = LdaModel(corpus=bow_corpus, id2word=dictionary,num_topics=10, passes=2)

In [27]:
for idx, topic in model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.009*"``" + 0.009*"''" + 0.008*"key" + 0.008*"space" + 0.006*"system" + 0.005*"ha" + 0.005*"information" + 0.005*"chip" + 0.005*"would" + 0.004*"clipper"
Topic: 1 
Words: 0.009*"..." + 0.008*"article" + 0.008*"wa" + 0.008*"would" + 0.007*"one" + 0.007*"like" + 0.007*"car" + 0.007*"get" + 0.007*".." + 0.006*"''"
Topic: 2 
Words: 0.018*"game" + 0.013*"team" + 0.010*"israel" + 0.008*"wa" + 0.008*"..." + 0.008*"university" + 0.007*"israeli" + 0.007*"player" + 0.007*"hockey" + 0.006*"year"
Topic: 3 
Words: 0.028*"wa" + 0.010*"``" + 0.010*"''" + 0.009*"would" + 0.007*"one" + 0.007*"think" + 0.006*"article" + 0.006*"..." + 0.006*"know" + 0.006*"time"
Topic: 4 
Words: 0.157*"''" + 0.108*"``" + 0.038*"q" + 0.035*"p" + 0.031*"r" + 0.030*"g" + 0.024*"b" + 0.023*"w" + 0.023*"u" + 0.022*"e"
Topic: 5 
Words: 0.009*"window" + 0.009*"drive" + 0.008*"card" + 0.008*"system" + 0.007*"university" + 0.006*"software" + 0.006*"nntp-posting-host" + 0.006*"use" + 0.006*"doe" + 0.006*"mac"
Top

In [24]:
bow_doc_4000 = bow_corpus[4000]
for i in range(len(bow_doc_4000)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4000[i][0], 
                                               dictionary[bow_doc_4000[i][0]], 
bow_doc_4000[i][1]))

Word 0 ("''") appears 2 time.
Word 2 ("``") appears 2 time.
Word 7 ("case") appears 1 time.
Word 32 ("nntp-posting-host") appears 1 time.
Word 38 ("point") appears 1 time.
Word 53 ("usa") appears 1 time.
Word 73 ("oh") appears 1 time.
Word 79 ("say") appears 1 time.
Word 106 ("fact") appears 1 time.
Word 120 ("like") appears 1 time.
Word 121 ("little") appears 1 time.
Word 139 ("steve") appears 1 time.
Word 194 ("people") appears 2 time.
Word 201 ("support") appears 1 time.
Word 204 ("would") appears 1 time.
Word 215 ("lot") appears 1 time.
Word 233 ("find") appears 1 time.
Word 268 ("fine") appears 1 time.
Word 281 ("joe") appears 1 time.
Word 297 ("order") appears 1 time.
Word 378 ("review") appears 1 time.
Word 407 ("look") appears 1 time.
Word 441 ("field") appears 1 time.
Word 466 ("consider") appears 1 time.
Word 468 ("done") appears 1 time.
Word 476 ("issue") appears 1 time.
Word 628 ("communication") appears 1 time.
Word 689 ("saying") appears 1 time.
Word 702 ("able") appears 

In [33]:
## Docuement 4000 of the Dataset is classified as such
for index, score in sorted(model[bow_corpus[4000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, model.print_topic(index, 10)))


Score: 0.40468040108680725	 
Topic: 0.020*"''" + 0.020*"``" + 0.010*"wa" + 0.009*"people" + 0.008*"would" + 0.008*"one" + 0.008*"god" + 0.005*"ha" + 0.005*"doe" + 0.005*"say"

Score: 0.3124648928642273	 
Topic: 0.009*"``" + 0.009*"''" + 0.008*"key" + 0.008*"space" + 0.006*"system" + 0.005*"ha" + 0.005*"information" + 0.005*"chip" + 0.005*"would" + 0.004*"clipper"

Score: 0.20847144722938538	 
Topic: 0.009*"``" + 0.009*"''" + 0.007*"article" + 0.007*"wa" + 0.007*"ha" + 0.007*"one" + 0.006*"cancer" + 0.006*"medical" + 0.006*"patient" + 0.006*"disease"

Score: 0.06799325346946716	 
Topic: 0.009*"..." + 0.008*"article" + 0.008*"wa" + 0.008*"would" + 0.007*"one" + 0.007*"like" + 0.007*"car" + 0.007*"get" + 0.007*".." + 0.006*"''"


### KNN 

In [34]:
dataset = df.clean_text
tfidf_vectoriser = TfidfVectorizer()
X = tfidf_vectoriser.fit_transform(dataset)

In [44]:
cat = set(df.categories)

In [45]:
cat

{'alt_atheism',
 'comp_graphics',
 'comp_os_ms-windows_misc',
 'comp_sys_ibm_pc_hardware',
 'comp_sys_mac_hardware',
 'comp_windows_x',
 'misc_forsale',
 'rec_autos',
 'rec_motorcycles',
 'rec_sport_baseball',
 'rec_sport_hockey',
 'sci_crypt',
 'sci_electronics',
 'sci_med',
 'sci_space',
 'soc_religion_christian',
 'talk_politics_guns',
 'talk_politics_mideast',
 'talk_politics_misc',
 'talk_religion_misc'}

In [47]:
Y = df.categories
Y

0        comp_sys_ibm_pc_hardware
1              talk_politics_misc
2                rec_sport_hockey
3           talk_politics_mideast
4              talk_politics_guns
                   ...           
18841       talk_politics_mideast
18842             rec_motorcycles
18843            rec_sport_hockey
18844                   rec_autos
18845               comp_graphics
Name: categories, Length: 18846, dtype: object

In [48]:
label_Y = LabelEncoder()
Y = label_Y.fit_transform(Y)
Y

array([ 3, 18, 10, ..., 10,  7,  1])

In [50]:
print(X.shape, Y.shape)

(18846, 121331) (18846,)


In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, shuffle=True)

In [58]:
N = X.shape[0]

In [60]:
k = np.sqrt(N)
k

137.2807342637706

In [61]:
knn = KNeighborsClassifier(n_neighbors=137)
knn.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=137)

In [62]:
pred = knn.predict(X_test)

In [63]:
## Report
print(classification_report(Y_test, pred))

              precision    recall  f1-score   support

           0       0.68      0.84      0.75       154
           1       0.75      0.69      0.72       198
           2       0.67      0.73      0.70       203
           3       0.58      0.71      0.64       201
           4       0.70      0.62      0.66       204
           5       0.89      0.69      0.78       196
           6       0.84      0.56      0.67       204
           7       0.81      0.86      0.84       190
           8       0.80      0.88      0.84       193
           9       0.91      0.86      0.89       213
          10       0.76      0.98      0.86       196
          11       0.81      0.91      0.85       200
          12       0.87      0.51      0.64       212
          13       0.93      0.77      0.84       188
          14       0.75      0.90      0.82       183
          15       0.75      0.92      0.83       206
          16       0.70      0.91      0.79       183
          17       0.85    